# Cancer Patient - Synopsis of Multiple Notes
    
#### Two Years After Coding Changes Sought to Decrease Documentation, Notes Remain ‘Bloated’

<br>
<div style="text-align: center;"><IMG SRC="Pics/Note_Bloat_Epic.png"  width="600" height="600" ALT="image">  
</div>
<br>


    
**Key Findings:**
- The average length of clinical notes has continued to increase over the last three years despite coding changes enacted in 2021 meant to reduce documentation burden.
- However, *time spent writing notes by providers has slightly decreased over the same time*.
- While overall average note length increased, *around 40% of providers reduced their average note length*.
- <span style="color:red">**1.7 billion**</span> clinical notes written by <span style="color:red">**166,318**</span> outpatient providers in the U.S. from May 2020 to April 2023 
- Average note length across all clinical notes has increased 8.1%
    - 4,628 characters in May 2020
    - 5,002 characters in April 2023  
    
#### Overview:
- Multiple office visit notes are read
- Compiled in a string (no particular chronological order)
- chatGPT parses and list chronologically with
    1. Clinical Findings
    2. Medications
    3. Lab Values
    


In [1]:
import os
import openai
import webbrowser
import datetime 
import re
from datetime import datetime
import subprocess


from chatGPT_utilities import setup_OpenAI_API
from chatGPT_utilities import json_to_dict
from chatGPT_utilities import get_completion
from chatGPT_utilities import get_completion_from_messages_FULL
from chatGPT_utilities import get_completion_from_messages_FULL_and_costs
from chatGPT_utilities import read_file
from chatGPT_utilities import token_usage
from chatGPT_utilities import print_token_usage
from chatGPT_utilities import write_to_file
from chatGPT_utilities import read_patient_notes

openai.api_key = setup_OpenAI_API()

# url_chatGPT_usage = 'https://platform.openai.com/account/usage'
# webbrowser.open_new(url_chatGPT_usage)

# Read clinical visit notes  

- Notes will not necessarily be retrieved in chronological order

![alt text](patient_notes_1.png "Here is Title")


In [2]:
starting_directory = os.getcwd()
# print(f'Starting Directory:\t{starting_directory}')
compilation_of_notes = read_patient_notes()
if os.getcwd() != starting_directory:
    os.chdir(starting_directory)
assert os.getcwd() == starting_directory , 'We should be in starting directory'



Note : The notes are not necessarily in chronological order


Processing the following notes:
  1: Nov_20.txt
  2: Oct_20.txt
  3: Sept_20.txt
  4: Aug_20.txt




			------------------------File Name: Nov_20.txt------------------------
Date: November 20


Following the completion of induction chemotherapy, he is experiencing a mix of physical and emotional symptoms. Today his ANC 900, HCT 30

Physically, he is still recovering from the side effects of chemotherapy. Reports fatigue, weakness, and a general sense of low energy. Nausea and vomiting gradually subsiding. Has reduced appetite, and has lost 2 kg since initial diagnosis. Emotionally, the patient feeling some uncertainty as awaiting the results of the treatment response evaluation and bone marrow assessment. Will continue to provide supportive care, address any ongoing symptoms, and offer emotional support prn.






			------------------------File Name: Oct_20.txt------------------------
Date: Oct 20
This 65-year-old white m

# Extract Diagnosis and Synopsis of Visits   

From the compilation of notes below, extract the information relevant to patient's condition. 

Patient's Diagnosis:  
Date of Diagnosis   :


List the dates of visits in chronological order and include:  
- Clinical findings
- Lab values   
- Medications


For each visit:
- Date  : 
    - Clinical Findings
    - Medications: 
    - HCT :
    - ANC : 


In [3]:
prompt_1 = f"""
Your task is to extract relevant information from 
clinical note. 

From the compilation of notes below, delimited by triple quotes extract the information relevant to patient's condition. 

Patient's Diagnosis:
Date of Diagnosis:


List the dates of visits in chronological order and 
include clinical findings, lab values and any medications 
(including any chemotherapy medications) mentioned during that visit
 in the following format:

Date # 1 - 
    Clinical Findings    
    Medications:
    Lab Values:

Date # 2 - 
    Clinical Findings   
    Medications:…
Date # N - 
    Clinical Findings
    Medications:



Review: ```{compilation_of_notes}```
    """


In [4]:
response = get_completion_from_messages_FULL_and_costs(prompt_1)

START QUERY
Patient's Diagnosis: Acute Myeloid Leukemia (AML)
Date of Diagnosis: August 2020

List of visits in chronological order:

Date # 1 - August 20
    Clinical Findings: Fatigue, lethargy
    Medications: statin, finasteride
    Lab Values: ANC 1500, HCT 27

Date # 2 - September 20
    Clinical Findings: Fatigue, lethargy
    Medications: cytarabine (Ara-C), daunorubicin
    Lab Values: Not mentioned

Date # 3 - October 20
    Clinical Findings: Fatigue, lethargy, decreased energy levels, mild nausea, occasional vomiting, decreased appetite, weight loss
    Medications: Arabinosylcytosine, midostaurin
    Lab Values: ANC 1250, HCT 29

Date # 4 - November 20
    Clinical Findings: Fatigue, weakness, low energy, nausea, vomiting subsiding, reduced appetite, weight loss
    Medications: Not mentioned
    Lab Values: ANC 900, HCT 30 


Time to Execute Task
Start  : 12:37:38
End    : 12:37:44
Elapsed: 6.535 seconds


Token Usage:
Cost to run query is based on token usage, which is $

# Extract only the ANC & HCT


Patient's Diagnosis:  
Date of Diagnosis:


For each visit:
- Date  : 
    - HCT :
    - ANC : 

In [5]:
prompt_2 = f"""
Your task is to extract relevant information from clinical note. 

From the compilation of notes below, delimited by triple quotes \
extract the information relevant to patient's condition. 

Patient's Diagnosis:
Date of Diagnosis  :


List the dates of visits in chronological order and include the lab values in the following format
Date # 1 : 
   HCT :
   ANC : 
    
Date # 2 : 
   HCT :
   ANC : 

Date # N :
   HCT :
   ANC : 

Do not list any additional information in the synopsis other than lab values.
If no lab values are listed for a given date, indicate 'N/A'

Review: ```{compilation_of_notes}```
"""

In [6]:
response = get_completion_from_messages_FULL_and_costs(prompt_2)

START QUERY
Patient's Diagnosis: Acute myeloid leukemia (AML)
Date of Diagnosis: August 2020

List of visits and lab values:

Date # 1: August 20
   HCT: 27
   ANC: 1500

Date # 2: September 20
   HCT: N/A
   ANC: N/A

Date # 3: October 20
   HCT: 29
   ANC: 1250

Date # 4: November 20
   HCT: 30
   ANC: 900 


Time to Execute Task
Start  : 12:37:44
End    : 12:37:49
Elapsed: 4.164 seconds


Token Usage:
Cost to run query is based on token usage, which is $0.002 / 1K tokens
Completion tokens:        118
Prompt tokens    :        738
Total tokens     :        856
Cost to run query: $0.002


# Summary Paragraph



Patient's Diagnosis:  
Date of Diagnosis:


For each visit:
- Date  : 
    - Summary :
    - HCT :
    - ANC : 

In [7]:
prompt_3 = f"""
As a medical chart abstractor, 
your task is to summarize the compilation of clinic notes.

From the compilation of notes below, delimited by triple quotes \
extract the information relevant to patient's condition. 

Patient's Diagnosis:
Date of Diagnosis  :


List the dates of visits in chronological order and include the lab values in the following format
Date # 1 : 
Summary :
    - Summation in bullet points
ANC:    
HCT:

Date # 2 : 
Summary :
    - Summation in bullet points
ANC:    
HCT:


Be sure to include the dates of each clinic visit
For each clinic visit, list the Summary first and the ANC HCT below that if it is mentioned
Be sure to list each visit in chronological order

Finally, include 5 blank lines and then create a heading "LAB VALUES"
and then list the HCT and ANC for each visit in a separate paragraph in the following
format

-------------------------------------
Date :
    ANC: 
    HCT: 


Review: ```{compilation_of_notes}```

"""

response = get_completion_from_messages_FULL_and_costs(prompt_3)

START QUERY
Patient's Diagnosis: Acute Myeloid Leukemia (AML)
Date of Diagnosis: August 2020

Dates of visits in chronological order and lab values:

Date #1: August 20
Summary:
- Patient presents with fatigue and lethargy
- Medications: statin and finasteride
ANC: 1500
HCT: 27

Date #2: September 20
Summary:
- Patient presented with fatigue and lethargy
- Diagnosed with AML
- Underwent induction chemotherapy (7+3 regimen)
- Close monitoring for treatment response and side effects
ANC: Not mentioned
HCT: Not mentioned

Date #3: October 20
Summary:
- Patient undergoing consolidation therapy (cycle 2 of 4)
- Experiencing common side effects of chemotherapy
- Close monitoring and supportive care provided
ANC: 1250
HCT: 29

Date #4: November 20
Summary:
- Patient experiencing physical and emotional symptoms after induction chemotherapy
- Fatigue, weakness, low energy, nausea, vomiting, reduced appetite, weight loss
- Waiting for treatment response evaluation and bone marrow assessment
ANC:

# Assemble Prompts in a List of Prompts

In [8]:
prompt_list = [prompt_1,prompt_2,prompt_3]

for prompt in prompt_list:
    relevant_details = prompt.split("\n\nReview:")[0]
    print(f'\033[91m\033[1m {relevant_details}\033[0m') 
    response = get_completion_from_messages_FULL_and_costs(prompt)

 
Your task is to extract relevant information from 
clinical note. 

From the compilation of notes below, delimited by triple quotes extract the information relevant to patient's condition. 

Patient's Diagnosis:
Date of Diagnosis:


List the dates of visits in chronological order and 
include clinical findings, lab values and any medications 
(including any chemotherapy medications) mentioned during that visit
 in the following format:

Date # 1 - 
    Clinical Findings    
    Medications:
    Lab Values:

Date # 2 - 
    Clinical Findings   
    Medications:…
Date # N - 
    Clinical Findings
    Medications:


START QUERY
Patient's Diagnosis: Acute Myeloid Leukemia (AML)
Date of Diagnosis: August 2020

List of visits in chronological order:

Date # 1 - August 20
    Clinical Findings: Fatigue, lethargy
    Medications: statin, finasteride
    Lab Values: ANC 1500, HCT 27

Date # 2 - September 20
    Clinical Findings: Fatigue, lethargy
    Medications: cytarabine (Ara-C), daunorub

# End